# **Penting**
- Jangan mengubah atau menambahkan cell text yang sudah disediakan, Anda hanya perlu mengerjakan cell code yang sudah disediakan.
- Pastikan seluruh kriteria memiliki output yang sesuai, karena jika tidak ada output dianggap tidak selesai.
- Misal, Anda menggunakan df = df.dropna() silakan gunakan df.isnull().sum() sebagai tanda sudah berhasil. Silakan sesuaikan seluruh output dengan perintah yang sudah disediakan.
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Pastikan Anda menggunakan variabel df dari awal sampai akhir dan tidak diperbolehkan mengganti nama variabel tersebut.
- Hapus simbol pagar (#) pada kode yang bertipe komentar jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan
- Pastikan Anda mengerjakan sesuai section yang sudah diberikan tanpa mengubah judul atau header yang disediakan.

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

import joblib
import warnings
warnings.filterwarnings('ignore')

# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [4]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
# Lengkapi kode berikut
# ___ = pd_read_csv("___.csv")

df = pd.read_csv("data_clustering.csv")

In [10]:
# Tampilkan 5 baris pertama dengan function head.
df.head()

,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,Target
0,14.09,1,36,0,70.0,0,81.0,1.0,5112.21,0
1,376.24,1,15,0,68.0,0,141.0,1.0,13758.91,6
2,126.29,1,23,2,19.0,3,56.0,1.0,1122.35,5
3,184.50,1,33,2,26.0,3,25.0,1.0,8569.06,5
4,13.45,0,1,2,45.0,3,198.0,1.0,7429.40,2


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TransactionAmount    2490 non-null   float64
 1   TransactionType      2516 non-null   int64  
 2   Location             2516 non-null   int64  
 3   Channel              2516 non-null   int64  
 4   CustomerAge          2516 non-null   float64
 5   CustomerOccupation   2516 non-null   int64  
 6   TransactionDuration  2516 non-null   float64
 7   LoginAttempts        2516 non-null   float64
 8   AccountBalance       2516 non-null   float64
 9   Target               2516 non-null   int64  
dtypes: float64(5), int64(5)
memory usage: 196.7 KB


# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [20]:
# Pisahkan fitur numerik dan kategorikal
numerical_features = df.select_dtypes(include=np.number).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Buat pipeline untuk preprocessing
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Buat pipeline clustering
cluster_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('cluster', KMeans(n_clusters=4, random_state=42, n_init=10)) # n_init=10 untuk menghindari warning
])

# Latih model clustering dan buat prediksi
df['Target'] = cluster_pipeline.fit_predict(df)

print("Kolom 'Target' berhasil dibuat dengan clustering.")
df.head()

Kolom 'Target' berhasil dibuat dengan clustering.


,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,Target
0,14.09,1,36,0,70.0,0,81.0,1.0,5112.21,0
1,376.24,1,15,0,68.0,0,141.0,1.0,13758.91,2
2,126.29,1,23,2,19.0,3,56.0,1.0,1122.35,1
3,184.50,1,33,2,26.0,3,25.0,1.0,8569.06,1
4,13.45,0,1,2,45.0,3,198.0,1.0,7429.40,0


In [21]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
# Definisikan fitur (X) dan target (y)
X = df.drop('Target', axis=1)
y = df['Target']

# Lakukan one-hot encoding pada fitur kategorikal di X
X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

# Bagi dataset menjadi 80% data latih dan 20% data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [25]:
# Inisialisasi dan latih model Decision Tree
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

# Lakukan prediksi pada data uji
y_pred_dt = decision_tree_model.predict(X_test)

# Tampilkan hasil evaluasi (akurasi, presisi, recall, F1-Score)
print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_dt))

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       201
           1       0.95      0.98      0.97       130
           2       0.91      0.91      0.91       149
           3       1.00      1.00      1.00        18

    accuracy                           0.94       498
   macro avg       0.95      0.95      0.95       498
weighted avg       0.94      0.94      0.94       498



In [26]:
# Menyimpan model Decision Tree
joblib.dump(decision_tree_model, 'decision_tree_model.h5')

['decision_tree_model.h5']

# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [ ]:
# Melatih model menggunakan algoritma klasifikasi scikit-learn selain Decision Tree.

In [ ]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.

In [ ]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu
# import joblib
# joblib.dump(___, 'explore_<Nama Algoritma>_classification.h5')

Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [27]:
# Tentukan rentang hyperparameter untuk diuji
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=3, # 3-fold cross-validation
                           n_jobs=-1, # Gunakan semua core CPU
                           verbose=2)

# Lakukan pencarian hyperparameter terbaik
grid_search.fit(X_train, y_train)

# Tampilkan hyperparameter terbaik
print(f"Best Parameters: {grid_search.best_params_}")

# Dapatkan model terbaik dari hasil tuning
best_dt_model = grid_search.best_estimator_

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2}


In [28]:
# Lakukan prediksi dengan model yang sudah di-tuning
y_pred_tuned = best_dt_model.predict(X_test)

# Tampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score
print("Tuned Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_tuned))

Tuned Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       201
           1       0.97      0.97      0.97       130
           2       0.92      0.91      0.92       149
           3       1.00      1.00      1.00        18

    accuracy                           0.94       498
   macro avg       0.96      0.96      0.96       498
weighted avg       0.94      0.94      0.94       498



In [30]:
# import joblib
# Menyimpan Model hasil tuning
joblib.dump(best_dt_model, 'tuning_classification.h5')

['tuning_classification.h5']

End of Code